# Pitcher Batter wOBAs 
The goal is:
* Given files that contain - one per record - each pitch thrown in the Statcase era (i.e., since 2017)
* Determine each batter's wOBA versus each pitcher. We'll use wOBA (for now) as a single value to gauge a batter's success against an individual pitcher.

# Configure Notebook
### Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import io

import boto3
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac

### HTML/CSS Styling for Markdown

In [2]:
%%html
<style>
.rendered_html tr, .rendered_html th, .rendered_html td {
  text-align: left;
}
# .rendered_html :first-child {
#   text-align: left;
# }
# .rendered_html :last-child {
#   text-align: left;
# }
</style>

# Build Data of Selected Pitch Events
Build a DataFrame with all of the pitch events that meet certain criteria (e.g., regular season, recorded by statcast) will be selected

### List Pitch Event Files
Load in all pitch event files from the selected period (based on the prefix variable).

* All files are available at [event files](https://s3.console.aws.amazon.com/s3/buckets/appleforge-merlin-develop-datalake/savant/?region=us-east-1&tab=overview)
* Files are in bucket name: appleforge-merlin-develop-datalake
* Path names are: savant/&lt;year&gt;-&lt;month&gt;-&lt;day&gt;.csv


In [4]:
bucket_name = 'appleforge-merlin-develop-datalake'
prefix = 'savant/2019'
s3_client = boto3.client('s3')
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(
    Bucket = bucket_name,
    Prefix = prefix
)
s3_objects = [key for page in pages for key in page['Contents']]
files = list(map(lambda o: o.get('Key'), s3_objects))
f"Pitch Event File Count: {len(files)}"

'Pitch Event File Count: 365'

### Build Data Frame of the Outcomes of all Plate Appearances

| Field | Description |
|:-------|:-------------|
| pitcher | MLB Player Id tied to the play event. |
| batter  | MLB Player Id tied to the play event. |
| events  | Event of the resulting Plate Appearance. (when not null) |

This is the fangraphs wOBA formula, but we're going to use an alternate version in which intentional base on balls (IBB) are treated as base on balls (BB).

![alt text](http://www.fangraphs.com/library/wp-content/uploads/2010/02/wOBA-flash-e1459706071159.png)


In [7]:
s3_session = boto3.session.Session().resource('s3')

def fetch_df(key: str) -> pd.DataFrame:
    zip_file_object = s3_session.Object(
        bucket_name=bucket_name,
        key=key
    )
    buffer = io.BytesIO(zip_file_object.get()["Body"].read())
    columns = ['pitcher','batter', 'events']
    df = pd.read_csv(
        filepath_or_buffer=buffer,
        header=0,
        sep=',',
        quotechar='"',
        usecols=columns,
        dtype={
            'pitcher': object,
            'batter': object,
            'pitch_name': object
        }
    )
    return df[~df['events'].isna()]

list_of_dataframes = list(map(fetch_df, files))
df = pd.concat(list_of_dataframes)
f"Outcome Event Count: {len(df)}"

'Outcome Event Count: 188865'

### Group by Pitcher/Batter

In [29]:
df['events'].unique()
all_event_names = set(df['events'].unique())
woba_event_names = {
 'double',
 'double_play',
 'field_error',
 'field_out',
 'fielders_choice',
 'fielders_choice_out',
 'force_out',
 'grounded_into_double_play',
 'hit_by_pitch',
 'home_run',
 'other_out',
 'sac_fly',
 'sac_fly_double_play',
 'single',
 'strikeout',
 'strikeout_double_play',
 'triple',
 'triple_play',
 'walk'}
available_event_names = all_event_names & woba_event_names
",".join(available_event_names)

'triple_play,hit_by_pitch,force_out,other_out,grounded_into_double_play,strikeout_double_play,triple,double,sac_fly,double_play,field_error,sac_fly_double_play,home_run,walk,strikeout,fielders_choice_out,single,fielders_choice,field_out'

In [27]:
woba_event_vector = df['events'].isin(available_event_names)
df_event_count = df[woba_event_vector].groupby(['pitcher', 'batter', 'events'])['events'].count()
df_event_count.head()

pitcher  batter  events   
282332   405395  field_out    2
                 single       3
         425844  field_out    1
                 strikeout    1
         429665  field_out    1
Name: events, dtype: int64

In [49]:
df_collapsed_event_count = df.copy()
df_collapsed_event_count['events'] = df_collapsed_event_count['events'].map({
    'triple_play': 'out', 
    'hit_by_pitch': 'hbp',
    'force_out': 'out', 
    'other_out': 'out', 
    'grounded_into_double_play': 'out', 
    'strikeout_double_play': 'out', 
    'triple': '3b',
    'double': '2b',
    'sac_fly': 'sf',
    'double_play': 'out', 
    'field_error': 'out', 
    'sac_fly_double_play': 'sf',
    'home_run': 'hr',
    'walk': 'bb',
    'strikeout': 'out', 
    'fielders_choice_out': 'out', 
    'single': '1b',
    'fielders_choice': 'out',
    'field_out': 'out'
})

def _woba(_1b, _2b, _3b, _bb, _hbp, _hr, _sf, _ab):
    return (0.69*_bb + 0.72*_hbp + 0.89*_1b + 1.27*_2b + 1.62*_3b + 2.10*_hr) / (_ab + _bb + _sf + _hbp)

df_collapsed_event_count = df_collapsed_event_count[woba_event_vector].groupby(['pitcher', 'batter', 'events'])['events'].count()
df_collapsed_event_count = df_collapsed_event_count.unstack().fillna(0)
df_collapsed_event_count['ab']=df_collapsed_event_count['1b']+df_collapsed_event_count['2b']+df_collapsed_event_count['3b']+df_collapsed_event_count['hr']+df_collapsed_event_count['out']
df_collapsed_event_count['woba'] = _woba(
    _1b = df_collapsed_event_count['1b'],
    _2b = df_collapsed_event_count['2b'],
    _3b = df_collapsed_event_count['3b'],
    _bb = df_collapsed_event_count['bb'],
    _hbp = df_collapsed_event_count['hbp'],
    _hr = df_collapsed_event_count['hr'],
    _sf = df_collapsed_event_count['sf'],
    _ab = df_collapsed_event_count['ab']
)
df_collapsed_event_count.head()

events           1b   2b   3b   bb  hbp   hr  out   sf   ab      woba
pitcher batter                                                       
282332  405395  3.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  5.0  0.534000
        425844  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  2.0  0.000000
        429665  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  2.0  0.526667
        430945  0.0  0.0  0.0  1.0  0.0  0.0  2.0  0.0  2.0  0.230000
        443558  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  2.0  1.050000

# Output
Build the output table and write it to S3

In [53]:
df_output = df_collapsed_event_count.copy()
df_output['event_count'] = df_output['ab'] + df_output['bb']+ df_output['sf'] + df_output['hbp']
df_output = df_output.drop(columns=['1b', '2b', '3b', 'bb', 'hbp', 'hr', 'out', 'sf', 'ab'])
df_output.head()

events              woba  event_count
pitcher batter                       
282332  405395  0.534000          5.0
        425844  0.000000          2.0
        429665  0.526667          3.0
        430945  0.230000          3.0
        443558  1.050000          2.0

In [54]:
import s3fs

def _write_out_df(df, key):
    bytes_to_write = df.to_csv(None).encode()
    fs = s3fs.S3FileSystem()
    with fs.open('s3://{bucket}/{key}'.format(
            bucket=bucket_name,
            key=key
        ), 'wb') as f:
        f.write(bytes_to_write)
        
_write_out_df(df_output, 'pitchtype/df_pitcher_batter_woba.csv')